In [ ]:
from scipy import optimize
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 4.2
k_BT = k_B * T


L_factor = 1
C_factor = 1
L_1, L_2 = 5e-12 * L_factor, 5e-12 * L_factor
C_1, C_2 = 500e-15 * C_factor, 500e-15 * C_factor                             # F
m_c, m_1, m_2 = C_1, C_1, C_2
x_c = PHI_0 / (2 * np.pi)
time_scale_factor = 1
t_c = np.sqrt(L_1 * C_1)
v_c = x_c / t_c

U0_1 = m_c * x_c**2 / t_c**2 / k_BT
U0_2 = m_2 * x_c**2 / t_c**2 / k_BT

In [ ]:
def detailed_pot_function(protocol, beta_1, d_beta, gamma_1, detail_info = False):
    U_ratio = 1

    beta_1 = beta_1
    beta_2 = beta_1
    d_beta_1 = d_beta
    d_beta_2 = d_beta
    _phi_1x = protocol["phi_1x"]
    _phi_2x = protocol["phi_2x"]
    _phi_1xdc = protocol["phi_1xdc"]
    _phi_2xdc = protocol["phi_2xdc"]
    _mu_12 = protocol["mu_12"]
    _xi = 1
    _phi_1dc = _phi_1xdc
    _phi_2dc = _phi_2xdc

    def Fcn(coord):
        _phi_1, _phi_2 = coord
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u1_3 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u1_4 = -d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)

        u2_1 = 1/2 * _xi * (_phi_2 - _phi_2x)**2
        u2_3 = beta_2 * np.cos(_phi_2) * np.cos(_phi_2dc/2)
        u2_4 = -d_beta_2 * np.sin(_phi_2) * np.sin(_phi_2dc/2)
        u3 = _mu_12 * _xi * (_phi_1 - _phi_1x) * (_phi_2 - _phi_2x)

        if detail_info:
            return {
                "u1_1": u1_1, "u1_3": u1_3, "u1_4": u1_4,
                "u2_1": u2_1, "u2_3": u2_3, "u1_4": u2_4,
                "u3": u3
            }
        else:
            return U_ratio * (u1_1 + u1_3 + u1_4 + u2_1 + u2_3 + u2_4 + u3)

    return Fcn


In [ ]:
def detailed_first_derivative_of_pot_function(protocol_2, beta, d_beta, gamma):
    beta_1 = beta
    beta_2 = beta
    d_beta_1 = d_beta
    d_beta_2 = d_beta

    _phi_1x = protocol_2["phi_1x"]
    _phi_2x = protocol_2["phi_2x"]
    _phi_1xdc = protocol_2["phi_1xdc"]
    _phi_2xdc = protocol_2["phi_2xdc"]
    _mu_12 = protocol_2["mu_12"]
    _xi = 1
    
    _phi_1dc = _phi_1xdc
    _phi_2dc = _phi_2xdc

    def Fcn(x):
        return [
            _xi * (x[0] - _phi_1x) - beta_1 * np.sin(x[0]) * np.cos(_phi_1xdc/2) + d_beta_1 * np.cos(x[0]) * np.sin(_phi_1xdc/2)
                    + _mu_12 * _xi * (x[1] - _phi_2x),
            _xi * (x[1] - _phi_2x) - beta_2 * np.sin(x[1]) * np.cos(_phi_2xdc/2) + d_beta_2 * np.cos(x[1]) * np.sin(_phi_2xdc/2)
                    + _mu_12 * _xi * (x[0] - _phi_1x)
        ]
    return Fcn

def detailed_find_all_critical_points_for_all_potential(protocol_2, beta, d_beta, gamma, guess = [(0, 0)]):
    """"
    To find all the critical points, including the saddle, minimum and maximum points, of the potential function
    """
    critical_points = [optimize.fsolve(detailed_first_derivative_of_pot_function(protocol_2, beta, d_beta, gamma), _g) for _g in guess]
    critical_potential = [detailed_pot_function(protocol_2, beta, d_beta, gamma)([x, y]) for x, y in critical_points]
    # energy_set = [coupled_flux_qubit_non_linear_approx_pot(sol[0], sol[1], _phi_1dcx, _phi_2dcx, _params_at_t) for sol in solution_set]
    return {"coord": critical_points, "potential": critical_potential}
    

    

In [ ]:

def cal_second_derivative(_phi_1, _phi_1xdc, beta_1 = 2.3):
    return 1 - beta_1 * np.cos(_phi_1) * np.cos(_phi_1xdc/2)

def get_barrier_height(protocol_2, beta_1, d_beta, gamma, protocol_duration, N):
    protocol_2["beta"] = beta_1
    protocol_2["d_beta"] = d_beta
    critical_dict = detailed_find_all_critical_points_for_all_potential(protocol_2, beta_1, d_beta, gamma, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
    critical_points = list(critical_dict.values())[0]

    critical_potential = [detailed_pot_function(protocol_2, beta_1, d_beta, gamma)([x, y]) for x, y in critical_points]
    
    delta_U17 = critical_potential[7] - critical_potential[1]
    delta_U37 = critical_potential[7] - critical_potential[3]

    second_derivative_array = np.array(cal_second_derivative(critical_points[1][0], protocol_2["phi_1xdc"], beta_1 = beta_1))
    plasma_freq = np.sqrt(second_derivative_array / (L_1 * C_1))
    escape_rate = plasma_freq / (2 * np.pi) * np.exp(-delta_U17 * U0_1)
    escape_event_per_operation = escape_rate * protocol_duration * np.sqrt(L_1 * C_1) * N

    result = {"coord": critical_points, "delta_U17": delta_U17 * U0_1, "delta_U37": delta_U37 * U0_1, "phi_1xdc": protocol_2["phi_1xdc"], "escape_rate": escape_rate, "escape_event_per_operation": escape_event_per_operation }
    return result


In [ ]:
mu_factor_h2 = 0.32
mu_factor_h2 = 0.4
beta_1 = 1.35

if beta_1 == 2.3:
    mu_12_factor_beta = 0.05
if beta_1 == 1.35:
    mu_12_factor_beta = 0.0765

protocol_harmonic_h2 = {
    "phi_1x": 0.1 * mu_factor_h2, "phi_2x": 0.0, "mu_12": mu_12_factor_beta * mu_factor_h2, 
    "phi_1xdc": 0.9,  "phi_2xdc": 0, "name": "h2", "color": "g", "linestyle": "-"
} 

get_barrier_height(protocol_harmonic_h2, beta_1, d_beta=0, gamma = 16, protocol_duration= 4, N = 1e12/2)

In [ ]:

protocol_harmonic_h2 = {
    "phi_1x": 0, "phi_2x": 0.0, "mu_12": 0, 
    "phi_1xdc": 0.8,  "phi_2xdc": 0, "name": "h2", "color": "g", "linestyle": "-"
} 

critical_points_and_escape_rate_info = get_barrier_height(protocol_harmonic_h2, 1.35, d_beta=0, gamma = 9, protocol_duration= 15, N = 1e12/2)

In [ ]:
phi_1xdc_array_1_35 = np.linspace(0, 1.1, 12)
phi_1xdc_array_2_3 = np.linspace(0, 2.1, 12)

def get_relation_between_barrier_height_and_critical_point(phi_1xdc_array, beta, gamma, protocol_duration = 15, N = 1e12):
    barrier_height_array = []
    critical_point_array = []
    escape_rate_array = []
    escape_event_per_operation_array = []
    for _phi_1xdc in phi_1xdc_array:
        _protocol = {
            "phi_1x": 0, "phi_2x": 0.0, "mu_12": 0, 
            "phi_1xdc": _phi_1xdc,  "phi_2xdc": 0, "name": "h2", "color": "g", "linestyle": "-"
        } 
        critical_points_and_escape_rate_info = get_barrier_height(_protocol, beta, d_beta=0, gamma = gamma, protocol_duration= protocol_duration, N = N/2)
        critical_point_array.append(critical_points_and_escape_rate_info['coord'][2])
        barrier_height_array.append(critical_points_and_escape_rate_info['delta_U17'])
        escape_rate_array.append(critical_points_and_escape_rate_info["escape_rate"])
        escape_event_per_operation_array.append(critical_points_and_escape_rate_info["escape_event_per_operation"])
    return critical_point_array, barrier_height_array, escape_rate_array, escape_event_per_operation_array

In [ ]:
critical_point_array_beta_1_35, barrier_height_array_beta_1_35, escape_rate_array_beta_1_35, escape_event_per_operation_array_beta_1_35 = get_relation_between_barrier_height_and_critical_point(phi_1xdc_array_1_35, beta = 1.35, gamma = 9, protocol_duration=60, N = 1.1e6)
critical_point_array_beta_2_3, barrier_height_array_beta_2_3, escape_rate_array_beta_2_3, escape_event_per_operation_array_beta_2_3 =  get_relation_between_barrier_height_and_critical_point(phi_1xdc_array_2_3, beta = 2.3, gamma = 9, protocol_duration=60, N = 1e6)

In [ ]:
plt.scatter(barrier_height_array_beta_1_35, np.array(critical_point_array_beta_1_35)[:, 0], label = r"$\beta = 1.35$")
plt.scatter(barrier_height_array_beta_2_3, np.array(critical_point_array_beta_2_3)[:, 0], label = r"$\beta = 2.3$")

In [ ]:
plt.plot(barrier_height_array_beta_1_35, escape_event_per_operation_array_beta_1_35)
# plt.yscale("log")
plt.ylim(top = 1e2)

# calculate the time for a complete oscillation